# Autograd    
`torch.autograd` 就是为方便用户使用，而专门开发的一套自动求导引擎，它能够根据输入和前向传播过程自动构建计算图，并执行反向传播。

计算图(Computation Graph)是现代深度学习框架如PyTorch和TensorFlow等的核心，其为高效自动求导算法——反向传播(Back Propogation)提供了理论支持，了解计算图在实际写程序过程中会有极大的帮助。

## 3.1 Autograd的前世今生       
深度学习的算法本质上是通过反向传播求导数，而PyTorch的**`autograd`**模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。
 
~~`autograd.Variable`是Autograd中的核心类，它简单封装了Tensor，并支持几乎所有Tensor有的操作。Tensor在被封装为Variable之后，可以调用它的`.backward`实现反向传播，自动计算所有梯度~~


~~Variable主要包含三个属性~~      
~~- `data`：保存Variable所包含的Tensor~~       
~~- `grad`：保存`data`对应的梯度，`grad`也是个Variable，而不是Tensor，它和`data`的形状一样~~        
~~- `grad_fn`：指向一个`Function`对象，这个`Function`用来反向传播计算输入的梯度~~             

---     


  *从0.4起, Variable 正式合并入Tensor, Variable 本来实现的自动微分功能，Tensor就能支持。读者还是可以使用Variable(tensor), 但是这个操作其实什么都没做。建议读者以后直接使用tensor*. 
  
  要想使得Tensor使用autograd功能，只需要设置**`tensor.requries_grad=True`**. 

## 3.2  requires_grad
PyTorch在autograd模块中实现了计算图的相关功能，autograd中的核心数据结构是Variable。从v0.4版本起，Variable和Tensor合并。我们可以认为需要求导(requires_grad)的tensor即Variable. autograd记录对tensor的操作记录用来构建计算图。

Variable提供了大部分tensor支持的函数，但其不支持部分`inplace`函数，因这些函数会修改tensor自身，而在反向传播中，variable需要缓存原来的tensor来计算反向传播梯度。如果想要计算各个Variable的梯度，只需调用根节点variable的`backward`方法，autograd会自动沿着计算图反向传播，计算每一个叶子节点的梯度。

***`variable.backward(gradient=None, retain_graph=None, create_graph=None)`***        
主要有如下参数：

- grad_variables：形状与variable一致，对于`y.backward()`，grad_variables相当于链式法则${dz \over dx}={dz \over dy} \times {dy \over dx}$中的$\textbf {dz} \over \textbf {dy}$。grad_variables也可以是tensor或序列。
- retain_graph：反向传播需要缓存一些中间结果，反向传播之后，这些缓存就被清空，可通过指定这个参数不清空缓存，用来多次反向传播。
- create_graph：对反向传播过程再次构建计算图，可通过`backward of backward`实现求高阶导数

In [0]:
import torch as t

In [2]:
a = t.randn(2, 3, requires_grad=True)

a = t.randn(2, 3).requires_grad_()

a = t.rand(2, 3)
a.requires_grad=True
a

tensor([[0.8913, 0.5391, 0.6613],
        [0.6033, 0.9297, 0.0086]], requires_grad=True)

In [0]:
b = t.zeros(2, 3, requires_grad=True) 

In [4]:
c = a.add(b)
c.grad_fn

In [5]:
d = c.sum()
print(d)
d.backward
print(d)

tensor(3.6332, grad_fn=<SumBackward0>)
tensor(3.6332, grad_fn=<SumBackward0>)


In [6]:
a.is_leaf, b.is_leaf, c.is_leaf, d.is_leaf

(True, True, False, False)

利用下面的函数，比对一下自动求导与手动求导：      
$$
y = x^2\bullet e^x
$$
它的导函数是：
$$
{dy \over dx} = 2x\bullet e^x + x^2 \bullet e^x
$$

In [0]:
def f(x):
    y = x** 2 * t.exp(x)
    return y

def gradf(x):
    dx = 2*x*t.exp(x) + x**2*t.exp(x)
    return dx

In [8]:
x = t.randn(3, 4, requires_grad=True)
y = f(x)
y

tensor([[2.0799e-02, 2.6712e-01, 2.8743e+01, 1.2769e-03],
        [8.3794e+00, 4.0024e-02, 2.3730e-01, 2.4196e-02],
        [2.9234e+00, 2.7208e-04, 3.1842e+01, 3.0278e+00]],
       grad_fn=<MulBackward0>)

In [9]:
y.backward(t.ones(y.size())) # gradient形状与y一致
x.grad

tensor([[ 3.2935e-01, -4.4218e-01,  5.7688e+01, -6.8901e-02],
        [ 2.0166e+01,  4.7840e-01,  1.4267e+00,  3.5864e-01],
        [ 8.6308e+00,  3.3533e-02,  6.3099e+01,  8.8707e+00]])

In [10]:
# 手动计算  
gradf(x)

tensor([[ 3.2935e-01, -4.4218e-01,  5.7688e+01, -6.8901e-02],
        [ 2.0166e+01,  4.7840e-01,  1.4267e+00,  3.5864e-01],
        [ 8.6308e+00,  3.3533e-02,  6.3099e+01,  8.8707e+00]],
       grad_fn=<AddBackward0>)

## 3.3 计算图     
PyTorch中`autograd`的底层采用了计算图，计算图是一种特殊的有向无环图（DAG），用于记录算子与变量之间的关系。一般用矩形表示算子，椭圆形表示变量。       
表达式 $ \textbf {z = wx + b}$可分解为 $\textbf{y = wx}$和 $\textbf{z = y + b}$，其计算图如图所示，图中`MUL`，`ADD`都是算子，$\textbf{w}$，$\textbf{x}$，$\textbf{b}$即变量。

![computation graph](img/com_graph.svg)

如上有向无环图中，$\textbf{X}$和$\textbf{b}$是叶子节点（leaf node），这些节点通常由用户自己创建，不依赖于其他变量。$\textbf{z}$称为根节点，是计算图的最终目标。利用链式法则很容易求得各个叶子节点的梯度。
$${\partial z \over \partial b} = 1,\space {\partial z \over \partial y} = 1\\
{\partial y \over \partial w }= x,{\partial y \over \partial x}= w\\
{\partial z \over \partial x}= {\partial z \over \partial y} {\partial y \over \partial x}=1 * w\\
{\partial z \over \partial w}= {\partial z \over \partial y} {\partial y \over \partial w}=1 * x\\
$$
而有了计算图，上述链式求导即可利用计算图的反向传播自动完成，如图所示。

![反向传播](img/com_graph_backward.svg)

在PyTorch实现中，autograd会随着用户的操作，记录生成当前variable的所有操作，并由此建立一个有向无环图。用户每进行一个操作，相应的计算图就会发生改变。更底层的实现中，图中记录了操作`Function`，每一个变量在图中的位置可通过其`grad_fn`属性在图中的位置推测得到。      
  
在反向传播过程中，autograd沿着这个图从当前变量（根节点$\textbf{z}$）溯源，可以利用链式求导法则计算所有叶子节点的梯度。每一个前向传播操作的函数都有与之对应的反向传播函数用来计算输入的各个variable的梯度，这些函数的函数名通常以`Backward`结尾

In [0]:
x = t.ones(1)
w = t.rand(1, requires_grad=True)
b = t.rand(1, requires_grad=True)

y = w * x
z = y + b

In [12]:
# y依赖于需要求导的w故为True
x.requires_grad, w.requires_grad, b.requires_grad, y.requires_grad, z.requires_grad

(False, True, True, True, True)

In [13]:
x.is_leaf, w.is_leaf, b.is_leaf, y.is_leaf, z.is_leaf

(True, True, True, False, False)

In [14]:
# grad_fn：查看variable的反向传播函数
z.grad_fn

In [15]:
# next_functions：保存grad_fn的输入，是一个tuple，其中的元素也是Function
# 第一个是y，它是乘法(mul)的输出，所以y.grad_fn是MulBackward
# 第二个是b，它是叶子节点，由用户创建，grad_fn为None，但是有
z.grad_fn.next_functions

((<MulBackward0 at 0x7f60ec56d7b8>, 0),
 (<AccumulateGrad at 0x7f60ec56d240>, 0))

In [16]:
z.grad_fn.next_functions[0][0] == y.grad_fn

True

In [17]:
# w, b
y.grad_fn.next_functions

((<AccumulateGrad at 0x7f60ec56d0b8>, 0), (None, 0))

In [18]:
w.grad_fn, x.grad_fn

(None, None)

In [19]:
z.backward(retain_graph=True)
w.grad

tensor([1.])

In [20]:
z.backward()
w.grad

tensor([2.])

PyTorch使用的是动态图，它的计算图在每次前向传播时都是从头开始构建，所以它能够使用Python控制语句（如for、if等）根据需求创建计算图。      
   
这点在自然语言处理领域中很有用，它意味着你不需要事先构建所有可能用到的图的路径，图在运行时才构建。

变量的`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都是True。这其实很好理解，对于$ \textbf{x}\to \textbf{y} \to \textbf{z}$，x.requires_grad = True，当需要计算$\partial z \over \partial x$时，根据链式法则，$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$，自然也需要求$ \frac{\partial z}{\partial y}$，所以`y.requires_grad` 会被自动标为True. 



有些时候我们可能不希望autograd对tensor求导。认为求导需要缓存许多中间结构，增加额外的内存/显存开销，那么我们可以关闭自动求导。对于不需要反向传播的情景（如inference，即测试推理时），关闭自动求导可实现一定程度的速度提升，并节省约一半显存，因其不需要分配空间计算梯度

In [21]:
w = t.rand(1, requires_grad=True)
x = t.ones(1, requires_grad=True)
y = w * x
w, x, y

(tensor([0.4142], requires_grad=True),
 tensor([1.], requires_grad=True),
 tensor([0.4142], grad_fn=<MulBackward0>))

### 设置不自动求导

In [22]:
with t.no_grad():
    w = t.rand(1, requires_grad=True)
    x = t.ones(1)
    y = w * x
# y依赖于w和x，但x和y
w, x, y    

(tensor([0.1583], requires_grad=True), tensor([1.]), tensor([0.1583]))

In [23]:
t.set_grad_enabled(False) 
w = t.rand(1, requires_grad=True)
x = t.ones(1)
y = w * x
# y依赖于w和x，但x和y
w, x, y

(tensor([0.0417], requires_grad=True), tensor([1.]), tensor([0.0417]))

In [24]:
t.set_grad_enabled(True)

### 修改Tensor数值    
如果我们希望对tensor，但是又不希望被记录, 可以使用 **`tensor.data` 或 `tensor.detach()`**

In [25]:
a = t.ones(3,4,requires_grad=True)
b = t.ones(3,4,requires_grad=True)
c = a * b  
a.data

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

In [26]:
a.data.requires_grad

False

In [27]:
d = a.data.sigmoid_()
d.requires_grad

False

In [28]:
a

tensor([[0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311, 0.7311]], requires_grad=True)

In [29]:
# tensor.datch()
# 近似于 tensor=a.data, 但是如果tensor被修改，backward可能会报错
tensor = a.detach()
tensor, tensor.requires_grad

(tensor([[0.7311, 0.7311, 0.7311, 0.7311],
         [0.7311, 0.7311, 0.7311, 0.7311],
         [0.7311, 0.7311, 0.7311, 0.7311]]), False)

In [30]:
# 统计tensor的一些指标，并不希望被记录
mean = tensor.mean()
std = tensor.std()
mean, std

(tensor(0.7311), tensor(0.))

### 查看变量梯度    
在反向传播过程中非叶子节点的导数计算完之后即被清空。若想查看这些变量的梯度，有两种方法：
- 使用autograd.grad函数
- 使用hook

`autograd.grad`和`hook`方法都是很强大的工具。     
推荐使用`hook`方法，但是在实际使用中应尽量避免修改grad的值。

In [31]:
w = t.rand(1, requires_grad=True)
x = t.ones(1, requires_grad=True)
y = w * x
z = y.sum()
w, x, y, z

(tensor([0.5200], requires_grad=True),
 tensor([1.], requires_grad=True),
 tensor([0.5200], grad_fn=<MulBackward0>),
 tensor(0.5200, grad_fn=<SumBackward0>))

In [32]:
# 非叶子节点grad计算完之后自动清空，y.grad是None
z.backward()
w.grad, x.grad, y.grad

(tensor([1.]), tensor([0.5200]), None)

In [33]:
# 方法一：
w = t.rand(3, requires_grad=True)
x = t.ones(3, requires_grad=True)
y = w * x
z = y.sum()
w, x, y, z

(tensor([0.5702, 0.0767, 0.8931], requires_grad=True),
 tensor([1., 1., 1.], requires_grad=True),
 tensor([0.5702, 0.0767, 0.8931], grad_fn=<MulBackward0>),
 tensor(1.5401, grad_fn=<SumBackward0>))

In [34]:
t.autograd.grad(z, y)

(tensor([1., 1., 1.]),)

In [35]:
# 方法二：使用hook
# hook是一个函数，输入是梯度，不应有返回值
def variable_hook(grad):
    print('y的梯度：', grad)
    
w = t.rand(3, requires_grad=True)
x = t.ones(3, requires_grad=True)
y = w * x
z = y.sum()

hook = y.register_hook(variable_hook)
z.backward()

# 若不重复使用hook，则移除
hook.remove()

y的梯度： tensor([1., 1., 1.])


关于 variable中grad属性和backward函数`grad_variables`参数的含义：

- variable $\textbf{x}$的梯度是目标函数${f(x)} $对$\textbf{x}$的梯度，$\frac{df(x)}{dx} = (\frac {df(x)}{dx_0},\frac {df(x)}{dx_1},...,\frac {df(x)}{dx_N})$，形状和$\textbf{x}$一致。
- 对于y.backward(grad_variables)中的grad_variables相当于链式求导法则中的$\frac{\partial z}{\partial x} = \frac{\partial z}{\partial y} \frac{\partial y}{\partial x}$中的$\frac{\partial z}{\partial y}$。z是目标函数，一般是一个标量，故而$\frac{\partial z}{\partial y}$的形状与variable $\textbf{y}$的形状一致。`z.backward()`在一定程度上等价于y.backward(grad_y)。`z.backward()`省略了grad_variables参数，是因为$z$是一个标量，而$\frac{\partial z}{\partial z} = 1$

In [36]:
x = t.Tensor([0,1,2]).requires_grad_()
y = x**2 + 2*x
z = y.sum()
print(x, y, z)

z.backward()
x.grad

tensor([0., 1., 2.], requires_grad=True) tensor([0., 3., 8.], grad_fn=<AddBackward0>) tensor(11., grad_fn=<SumBackward0>)


tensor([2., 4., 6.])

In [37]:
x = t.Tensor([0,1,2]).requires_grad_()
y = x**2 + 2*x
z = y.sum()

y_gredient = t.Tensor([1, 1, 1]) # dz/dy
y.backward(y_gredient)
x.grad

tensor([2., 4., 6.])

另外值得注意的是，只有对variable的操作才能使用autograd，如果对variable的data直接进行操作，将无法使用反向传播。除了对参数初始化，一般我们不会修改variable.data的值

### 计算图的特点：
- autograd根据用户对variable的操作构建其计算图。对变量的操作抽象为`Function`。
- 对于那些不是任何函数(Function)的输出，由用户创建的节点称为叶子节点，叶子节点的`grad_fn`为None。叶子节点中需要求导的variable，具有`AccumulateGrad`标识，因其梯度是累加的。
- variable默认是不需要求导的，即`requires_grad`属性默认为False，如果某一个节点requires_grad被设置为True，那么所有依赖它的节点`requires_grad`都为True。
- variable的`volatile`属性默认为False，如果某一个variable的`volatile`属性被设为True，那么所有依赖它的节点`volatile`属性都为True。volatile属性为True的节点不会求导，volatile的优先级比`requires_grad`高。
- 多次反向传播时，梯度是累加的。反向传播的中间缓存会被清空，为进行多次反向传播需指定`retain_graph`=True来保存这些缓存。
- 非叶子节点的梯度计算完之后即被清空，可以使用`autograd.grad`或`hook`技术获取非叶子节点的值。
- variable的grad与data形状一致，应避免直接修改variable.data，因为对data的直接操作无法利用autograd进行反向传播
- 反向传播函数`backward`的参数`grad_variables`可以看成链式求导的中间结果，如果是标量，可以省略，默认为1
- PyTorch采用动态图设计，可以很方便地查看中间层的输出，动态的设计计算图结构

## 3.4 Autograd拓展         

目前绝大多数函数都可以使用`autograd`实现反向求导，但如果需要自己写一个复杂的函数，不支持自动反向求导怎么办? 写一个`Function`，实现它的前向传播和反向传播代码，`Function`对应于计算图中的矩形， 它接收参数，计算并返回结果。     


---


- 自定义的Function需要继承autograd.Function，没有构造函数`__init__`，forward和backward函数都是静态方法
- backward函数的输出和forward函数的输入一一对应，backward函数的输入和forward函数的输出一一对应
- backward函数的grad_output参数即t.autograd.backward中的`grad_variables`
- 如果某一个输入不需要求导，直接返回None，如forward中的输入参数x_requires_grad显然无法对它求导，直接返回None即可
- 反向传播可能需要利用前向传播的某些中间结果，需要进行保存，否则前向传播结束后这些对象即被释放

Function的使用利用Function.apply(variable)